## CIE5603: Advanced Projects on GRS

In [1]:
# Creating video 
# Importing all the libraries 
import numpy as np
import rasterio
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.colorbar as colorbar
from datetime import date, datetime
import matplotlib.colors as colors
import matplotlib.animation as animation
from matplotlib import rcParams
plt.rc('font', family='serif')
# plt.rc('xtick', labelsize='x-small')
# plt.rc('ytick', labelsize='x-small')
# plt.rcParams.update({'font.size': 12})

In [2]:
path = 'data/'
file_in = 'merged_layer_2000_to_2019.tif'

# read in file with geotiff geographic information
src = rasterio.open(path + file_in)

# print out metadata information
for k in src.meta:
  print(k,src.meta[k])

transform = src.transform
N = src.width
M = src.height
dx = transform.a
dy = transform.e
minx = transform.c
maxy = transform.f
dy = - dy
# Read the image data, flip upside down if necessary
LST = src.read()
# if dy < 0:
#   dy = -dy
#   LST = np.flip(LST, 0)
LST[:, :, :] = LST[:, :, :]/50
print(LST.shape)

# print(LST[18,1000,3000])

driver GTiff
dtype float32
nodata None
width 4305
height 2845
count 20
crs EPSG:4326
transform | 0.00, 0.00, 3.36|
| 0.00,-0.00, 53.55|
| 0.00, 0.00, 1.00|
(20, 2845, 4305)


In [3]:
ndates = LST.shape[0]
# Generate X and Y grid locations
x = minx + dx / 2 + dx * np.arange(N)
y = maxy - dy / 2 - dy * np.arange(M - 1, -1, -1)

In [13]:
# unfiltered LST data
%matplotlib qt
extent = [x[0], x[-1], y[0] , y[-1]]
plt.figure(figsize=(8, 8))
index = 17
fig = plt.imshow(LST[index, :, :], extent=extent, cmap='jet')
# plt.title(
#     'LST ' + datetime.fromordinal(tsdates[:, index] - 366).strftime('%Y-%m-%d') + ' DAY ' + str(int(ts[:, index])))
plt.colorbar(label='LST [K]')
plt.clim(280, 300)
# Hide the axes and remove the space around them
# plt.axis('off')
plt.ylabel('Latitude [o]')
plt.xlabel('Longitude [o]')
plt.axis('tight')
plt.show()


In [16]:
import cmocean
frames = [] # for storing the generated images
# Using matplotlib qt in order to see the video
%matplotlib qt
fig, ax=plt.subplots()
for i in range(ndates):
    if i < 10:
        datenum = '200' + str(i)
    else: 
        datenum = '20' + str(i)
    im = plt.imshow(LST[i, :, :], extent=extent, cmap=cmocean.cm.thermal, animated=True)
    title = ax.text(0.5,1.05,"Land Surface Temperature (LST) for year " + datenum, 
                        size=plt.rcParams["axes.titlesize"],
                        ha="center", transform=ax.transAxes, ) 
    frames.append([im, title])
#     frames.append([plt.imshow(LST[i, :, :], extent=extent, cmap='jet', animated=True)])
    plt.clim(280, 300)
plt.colorbar(label='LST [K]')
# Hide the axes and remove the space around them
# plt.axis('off')
plt.axis('tight')
plt.ylabel(r'Latitude [$\degree$]')
plt.xlabel(r'Longitude [$\degree$]')
ani = animation.ArtistAnimation(fig, frames, interval=1000, blit=False, repeat=False)
# Uncomment below line to save the video
# ani.save('results/LST_new_for_20_years.gif')
plt.show()

<div class='alert alert-warning'>
    Run the script below for clipping raster with shapefile and saving them, otherwise move to next cell
</div>


In [ ]:
# Clipping raster into the shapefile and saving them
import fiona
import rasterio
import rasterio.mask

with fiona.open("shapefile/NLD_adm0.shp", "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]

with rasterio.open("data/LST_2010.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, features,
                                                  crop=True)
    out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("data/LST_2010_masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)

# Merging the layers GDAL command 
# python3 -m gdal_merge -separate -ot Float32 -of GTiff -o "C:/Users/ullas/Desktop/Advance Projects/merged_layer_2000_to_2019.tif" --optfile C:/Users/ullas/AppData/Local/Temp/processing_13e88a7c848f46d6aea19e794d2feff8/fbb94262af15442ca19231163256fbfe/mergeInputFiles.txt

In [ ]:
# 